# Data Cleaning

In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import geopandas
import folium
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
%matplotlib inline

In [69]:
df = pd.read_csv('kc_house_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB


It appears that most categories are not missing any observations.


In [70]:
df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

Waterfront has 2376 null values (use mapping to explore these)
View has 63 null values. Could possibly just get rid of these as they are a small number
And yr_renovated has 3842 null values (indicates that it has not been renovated)

# Creating New Variables

Creating a new variable of house age might be more useful. Can do this by doing 2015 - yr_built

In [71]:
df['house_age'] = df.yr_built.map(lambda x: 2015 - x)
df.head()
# now a new category has been added to the data set which utilises a lambda function 

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,house_age
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,60
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,64
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,82
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,50
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,28


Might also be worth using renovation as a dummy variable. Give it a value of 1 if it has been renovated, 0 if not. 
This may be useful in our analysis. i.e. does a house being renovated increase house price. 

In [72]:
df['renovated'] = df.yr_renovated.map(lambda x: 1 if x > 0 else 0)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,house_age,renovated
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,60,0
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,64,1
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,82,0
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,50,0
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,28,0


# Dropping Outliers

In [73]:
display(df.max())
df.min()
# there appears to be a property with 33 bedrooms, as this seems very high, it may be worth checking

id               9900000190
date               9/9/2014
price               7.7e+06
bedrooms                 33
bathrooms                 8
sqft_living           13540
sqft_lot            1651359
floors                  3.5
waterfront                1
view                      4
condition                 5
grade                    13
sqft_above             9410
sqft_basement             ?
yr_built               2015
yr_renovated           2015
zipcode               98199
lat                 47.7776
long               -121.315
sqft_living15          6210
sqft_lot15           871200
house_age               115
renovated                 1
dtype: object

id                 1000102
date             1/10/2015
price                78000
bedrooms                 1
bathrooms              0.5
sqft_living            370
sqft_lot               520
floors                   1
waterfront               0
view                     0
condition                1
grade                    3
sqft_above             370
sqft_basement          0.0
yr_built              1900
yr_renovated             0
zipcode              98001
lat                47.1559
long              -122.519
sqft_living15          399
sqft_lot15             651
house_age                0
renovated                0
dtype: object

In [74]:
df.loc[df['bedrooms'] == 33]
# it appears that this 33 bedrooms house only has 1.75 bathrooms, 1620 living space. This is not feasible and is likely to be a typo. We could change the value of bedrooms to 3

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,house_age,renovated
15856,2402100895,6/25/2014,640000.0,33,1.75,1620,6000,1.0,0.0,0.0,...,580.0,1947,0.0,98103,47.6878,-122.331,1330,4700,68,0


In [75]:
df.bedrooms = df.bedrooms.replace([33], 3)
# this line replaces that observation's bedroom from 33 to 3
# can check by locating by index number 15856
df.iloc[15856]

id               2402100895
date              6/25/2014
price                640000
bedrooms                  3
bathrooms              1.75
sqft_living            1620
sqft_lot               6000
floors                    1
waterfront                0
view                      0
condition                 5
grade                     7
sqft_above             1040
sqft_basement         580.0
yr_built               1947
yr_renovated              0
zipcode               98103
lat                 47.6878
long               -122.331
sqft_living15          1330
sqft_lot15             4700
house_age                68
renovated                 0
Name: 15856, dtype: object

It also looks like several properties have 0.5 bathrooms. This is pretty unfeasible and may indicate student housing etc which we are not interested in.
We will also drop these from the dataframe

In [76]:
df = df.loc[~(df['bathrooms'] < 1)]
df.min()

id                 1000102
date             1/10/2015
price                78000
bedrooms                 1
bathrooms                1
sqft_living            390
sqft_lot               520
floors                   1
waterfront               0
view                     0
condition                1
grade                    4
sqft_above             390
sqft_basement          0.0
yr_built              1900
yr_renovated             0
zipcode              98001
lat                47.1559
long              -122.519
sqft_living15          399
sqft_lot15             651
house_age                0
renovated                0
dtype: object

View is an index which should contain a value, a null value indicates missing data. Seeing as there are only 63 missing observations, as this doesn't represent a large proportion of the data set, we will also be dropping these

In [77]:
pd.unique(df['view'].values.ravel())

array([ 0., nan,  3.,  4.,  2.,  1.])

In [78]:
df.view.dropna(inplace = True)
pd.unique(df['view'].values.ravel())

array([0., 3., 4., 2., 1.])

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21522 entries, 0 to 21595
Data columns (total 23 columns):
id               21522 non-null int64
date             21522 non-null object
price            21522 non-null float64
bedrooms         21522 non-null int64
bathrooms        21522 non-null float64
sqft_living      21522 non-null int64
sqft_lot         21522 non-null int64
floors           21522 non-null float64
waterfront       19152 non-null float64
view             21459 non-null float64
condition        21522 non-null int64
grade            21522 non-null int64
sqft_above       21522 non-null int64
sqft_basement    21522 non-null object
yr_built         21522 non-null int64
yr_renovated     17696 non-null float64
zipcode          21522 non-null int64
lat              21522 non-null float64
long             21522 non-null float64
sqft_living15    21522 non-null int64
sqft_lot15       21522 non-null int64
house_age        21522 non-null int64
renovated        21522 non-null int64

In [ ]:
# Ok can't seem to do this so ask tomorrow
